# Reading soiling data

Notebook containing basic functionality on NREL soiling data: https://datahub.duramat.org/dataset/data-for-validating-models-for-pv-module-performance

In [1]:
import numpy as np
import os, sys
import math
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import pandas as pd
from ast import literal_eval


## Read data
Read input time series. Manual washing dates are given as input. 

In [2]:
def parse_csv(filename):
    df_meta = pd.read_csv(filename, nrows=1)
    df_temp = pd.read_csv(filename, sep='\n', header=None, names=["temp"])
    col_names = df_temp.temp.iloc[2].split(',')
    col_names.extend(['I-V Curve I Values', 'I-V Curve V Values'])
    rows = []
    num_r = df_temp.iloc[3:].shape[0]
    
    # handle columns
    for i in range(num_r):
        data = df_temp.temp.loc[3+i].split(',')
        N = int(data[41])
        rows.append(data[:42])
        rows[i].extend([np.array(data[42:42+N]), np.array(data[42+N:])])
    df = pd.DataFrame(columns=col_names, data=rows)
    
    # convert to datetime
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], errors='coerce') 
    
    # convert to datetime.time (99:99 will be replaced with nan)
    df.iloc[:, 38] = pd.to_datetime(df.iloc[:, 38], format= '%H:%M', errors='coerce').dt.time
    df.iloc[:, 39] = pd.to_datetime(df.iloc[:, 39], format= '%H:%M', errors='coerce').dt.time
    
    # infer and convert to appropriate types for each column
    idx = [0, 38, 39, 42, 43]
    for j in range(1, df.shape[1]):
        if j in idx:
            continue
        df.iloc[:, j] = df.iloc[:, j].apply(literal_eval)
        
    # replace -9999 precipitation with nan
    idx1 = [x for x in list(range(df.shape[1])) if x not in idx]
    df.iloc[:, idx1] = df.iloc[:, idx1].replace(-9999, np.nan)
    
    return df_meta, df

Define filename and manual washing dates:

In [3]:
#use dataset named Eugene_mSi0188.csv
filename = '/data/data1/data-for-validating-models/Data For Validating Models/Eugene/Eugene_mSi0188.csv'

#manual washing dates for Eugene: 
dates_wash_start = pd.to_datetime(pd.Series(['2013-03-11 00:00:00', '2013-07-10 00:00:00', '2013-08-14 00:00:00', '2013-08-21 00:00:00', '2013-08-26 00:00:00']))
dates_wash_stop = pd.to_datetime(pd.Series(['2013-03-12 00:00:00', '2013-07-11 00:00:00', '2013-08-15 00:00:00', '2013-08-22 00:00:00','2013-08-27 00:00:00']))

Parse csv file. Omit redundant columns.

In [4]:
df_meta, df = parse_csv(filename)

df = df.iloc[:, [0, 1, 3, 5, 7, 9, 11, 13, 22, 26, 27, 30, 33, 37]]

df.columns = ['timestamp', 'irradiance', 'mod_temp', 'Shortcircuitcurrent', 'power',
               'dcurrent','Voltageatmax',
               'Opencircuitvoltage', 'humidity', 'precipitation', 'dni', 'ghi', 'dhi', 'soiling_derate']

df = df.set_index('timestamp')
df = df.dropna()
df.head()

,irradiance,mod_temp,Shortcircuitcurrent,power,dcurrent,Voltageatmax,Opencircuitvoltage,humidity,precipitation,dni,ghi,dhi,soiling_derate
timestamp,,,,,,,,,,,,,
2012-12-20 10:40:02,22.8,4.5,0.0688,0.8035,0.0571,14.0619,18.7348,86.1,18.8,0.0,28.0,28.3,1.0
2012-12-20 10:45:02,31.0,4.6,0.0930,1.1519,0.0780,14.7685,19.3450,85.4,18.8,0.1,37.1,37.6,1.0
2012-12-20 10:50:02,34.0,4.6,0.1017,1.2762,0.0853,14.9553,19.5073,86.0,18.8,0.1,41.2,41.8,1.0
2012-12-20 10:55:02,29.2,4.6,0.0875,1.0675,0.0730,14.6176,19.2116,85.9,18.8,0.1,35.4,35.9,1.0
2012-12-20 11:00:02,32.9,4.6,0.0979,1.2187,0.0818,14.9050,19.4271,85.1,18.9,0.1,40.5,41.1,1.0


## Rescale (if needed)
Use MinMaxScaler to rescale all points to [0,1].

In [5]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

In [6]:
df_scaled.describe()

,irradiance,mod_temp,Shortcircuitcurrent,power,dcurrent,Voltageatmax,Opencircuitvoltage,humidity,precipitation,dni,ghi,dhi,soiling_derate
count,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000,41670.000000
mean,0.240040,0.488759,0.242939,0.229354,0.239599,0.552331,0.606761,0.623341,0.027583,0.339249,0.237523,0.129672,0.906683
std,0.236484,0.176826,0.238951,0.227015,0.238184,0.140278,0.144980,0.225187,0.089349,0.391078,0.209455,0.107905,0.189237
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.045438,0.349922,0.047995,0.042359,0.045305,0.493751,0.539460,0.446250,0.000000,0.004215,0.068020,0.056834,0.861111
50%,0.136528,0.461897,0.136770,0.128678,0.132569,0.587347,0.635596,0.653750,0.000000,0.026693,0.159569,0.089934,1.000000
75%,0.419793,0.622084,0.420410,0.405196,0.417268,0.645686,0.702377,0.815000,0.004184,0.792875,0.369035,0.169876,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Extract rains (if needed)
Use precipitation values to extract rains. Each rain corresponds to a maximal period of positive precipitation.

In [7]:
if df.precipitation.iloc[0]>0:
    precipitation = pd.concat([pd.Series({min(df.index)-pd.Timedelta('1s'): 0}),df.precipitation])
else:
    precipitation = df.precipitation

precipitation.index = pd.to_datetime(precipitation.index)
df_dates = pd.DataFrame(index = precipitation.index)
df_dates["rain_start"] = precipitation[(precipitation.shift(-1) > 0) & (precipitation == 0)] # compare current to next
df_dates["rain_stop"] = precipitation[(precipitation.shift(1) > 0) & (precipitation == 0)] # compare current to prev
dates_rain_start = pd.Series(df_dates.rain_start.index[df_dates.rain_start.notna()])
dates_rain_stop = pd.Series(df_dates.rain_stop.index[df_dates.rain_stop.notna()])

Number of rains:

In [8]:
dates_rain_stop.shape[0]

69